# Deprecated cells

## Treinando um Identificador de Entidades a partir do DHBB
Identificadores de entidades são algoritmos treinados em corpora manualmente anotados. Como necessitamos reconhecer outros tipos de entidades, precisamos retreinar o modeld usando o próprio DHBB. Para este fim utilizaremos os dicionários já disponíveis no DHBB,  juntamente com o índice construído no capítulo 2 para recuperar o contexto de cada entrada dos dicionários.

In [13]:
!python3 -m spacy download pt_core_news_lg

2022-11-26 11:54:35.372031: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:
2022-11-26 11:54:35.372088: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:
2022-11-26 11:54:35.372095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-11-26 11:54:36.506108: W tensorflow/compiler/xla/stream_executor/platform/default/ds

In [14]:
from whoosh import index 
import os
from whoosh.qparser import QueryParser
from whoosh import qparser

O primeiro passo é abrirmos o nosso indice.

In [15]:
if os.path.exists('indexdir'):
    indice = index.open_dir('indexdir')

In [16]:
indice.doc_count()

26896

In [53]:
def busca(consulta):
    qp = QueryParser("corpo", indice.schema)
    qp.add_plugin(qparser.EveryPlugin())
    query = qp.parse(consulta)
    
    with indice.searcher() as searcher:
        results = [(dict(hit),hit.highlights("corpo")) for hit in searcher.search(query, limit=10)]
    return results

In [68]:
resultados = busca('"filho de"')
# print(resultados,type(resultados[0]), len(resultados[0]))
# type(resultados)
print('Resultados:', len(resultados))
for res in resultados[:10]:
    print(res[0]['title'])
    display(HTML(res[1]))
    print()

Resultados: 10
NOGUEIRA FILHO, Paulo



NOGUEIRA FILHO, Paulo



NOGUEIRA FILHO, Paulo



NOGUEIRA FILHO, Paulo



SIMÕES FILHO, Ernesto



SIMÕES FILHO, Ernesto



SIMÕES FILHO, Ernesto



SIMÕES FILHO, Ernesto



MARCONDES FILHO



MARCONDES FILHO


Agora já temos os ingredientes necessários para treinar um modelo de entidades usando a biblioteca spacy.

In [19]:
import random, os
from tqdm import tqdm
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [20]:
for verb in biograficos.itertuples():
    print(verb.title)
    break

PARTIDO POPULAR DO DISTRITO FEDERAL


Primeiro precisamos criar o conjunto de treinamento do modelo. e deve ter  a forma de uma lista como a descrita abaixo.
```python
    TRAIN_DATA = [
        ("nasceu em Itaporanga ( PB ) no dia 29 de dezembro de 1944 , filho de Argemiro Abílio de Sousa", 
         {"entities": [(10, 20, "LOC"), (69, 93, "PERSON")]}
        ),
    ]
```

Como primeira abordagem de treinamento, vamos atualizar o modelo que vem com o spacy adicionando mais um tipo de entidade: "eventos". Posteriormente, o modelo pode continuar a ser incrementado adicionando-se outros tipos de entidade. Vamos também aproveitar as entidades reconhecidas pelo modelo base no corpus do DHBB para reforçar o treinamento do modelo no contexto linguístico do DHBB.

In [43]:
from spacy.tokens import DocBin

path = "dhbb/dic/evento.txt"
with open(path, 'r', encoding='utf-8') as f:
    dicio = f.readlines()

def gera_dados_treinamento(dicionário, tag):
    db = DocBin() # Armazena os exemplos para treinamento em um objeto DocBin
    for verb in tqdm(biograficos.itertuples()):
        texto = verb.corpo
        doc = nlp(texto)
        entidades = []#(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        for evento in dicionário:
            try:
                posini = texto.index(evento)
            except ValueError:
                continue
            posfim = posini + len(evento)
            span = doc.char_span(posini,posfim,tag)
            if span is not None:
                entidades.append(span)
        doc.ents = entidades
        db.add(doc)
    db.to_disk('train.spacy')
    return db
            
            

In [45]:
dados = gera_dados_treinamento(dicio, "EVT")
dados

1845it [03:12,  9.60it/s]


ValueError: [E1010] Unable to set entity information for token 114 which is included in more than one span in entities, blocked, missing or outside.

In [55]:
if os.path.exists('train.spacy'):
    print("Carregando dados de treinamento do disco...")
    dados = DocBin().from_disk('train.spacy')
    dados = pickle.load(open('ner_training.pickle','rb'))
else:
    dados = gera_dados_treinamento(dicio, "EVT")
    # pickle.dump(dados,open('ner_training.pickle','wb'))

0it [00:00, ?it/s]


TypeError: an integer is required

Agora temos o tipo `EVT`, evento dentre as categorias reconhecidas.

In [41]:
set([e[2] for d in dados for e in d[1]['entities']])

{'EVT', 'LOC', 'MISC', 'ORG', 'PER'}

Agora reservamos 1000 documentos para testar o reconhecimento de nossa nova categoria, usando o restante para treinar o modelo.

In [42]:
TEST_DATA = dados[-1000:]
TRAIN_DATA = dados[:1000]

In [43]:
def main(model="pt_core_news_sm", output_dir="dhbb_nlp", n_iter=25):
    """Carrega o modelo, configura a pipeline e treina o NER."""
    if model is not None:
        nlp = spacy.load(model)  
        print(f"Modelo {model} carregado")
    else:
        nlp = spacy.blank("pt")  # cria um novo modelo, vazio.
        print("Modelo vazio criado para 'pt'")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [44]:
main()

Modelo pt_core_news_sm carregado


ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 